In [12]:
import pymongo
from pymongo import MongoClient

# Verbindung zu MongoDB herstellen
client = MongoClient("mongodb://localhost:27017/")

# Zugriff auf die Datenbank und Sammlung
db = client["fingerprintDB"]
collection = db["fingerprints"]

# Daten für User 2 und User 3 abrufen
user2_data = collection.find_one({"username": "username_2"})
user3_data = collection.find_one({"username": "username_3"})

# Daten für andere Benutzer abrufen (negative Beispiele)
negative_data = collection.find({"username": {"$nin": ["username_2", "username_3"]}})


In [13]:
import pymongo
from pymongo import MongoClient
import base64
import numpy as np
from PIL import Image
from io import BytesIO
from sklearn.model_selection import train_test_split


def decode_base64_image(base64_string):
    base64_data = base64_string.split(",")[1]
    byte_data = base64.b64decode(base64_data)
    image_data = BytesIO(byte_data)
    image = Image.open(image_data)
    return np.array(image)

def get_canvases(data):
    return [decode_base64_image(canvas) for canvas in data["canvases"]]


In [14]:
# Canvases für User 2 und User 3
user2_canvases = get_canvases(user2_data)
user3_canvases = get_canvases(user3_data)

# Positive Beispiele (User 2)
X_positive = np.array(user2_canvases)
y_positive = np.ones(len(X_positive))

# Negative Beispiele (User 3)
X_negative = np.array(user3_canvases)
y_negative = np.zeros(len(X_negative))

# Kombinieren der Daten
X = np.concatenate((X_positive, X_negative), axis=0)
y = np.concatenate((y_positive, y_negative), axis=0)

# Aufteilen der Daten in Trainings- und Testsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binäre Klassifikation
    return model

input_shape = (35, 280, 4)  # Beispielhafte Eingabeform (Höhe, Breite, Kanäle)
model = create_cnn_model(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [16]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
95/95 [==============================] - 32s 320ms/step - loss: 12.7292 - accuracy: 0.7563 - val_loss: 0.0568 - val_accuracy: 0.9816
Epoch 2/10
95/95 [==============================] - 31s 324ms/step - loss: 0.0249 - accuracy: 0.9937 - val_loss: 0.0147 - val_accuracy: 0.9961
Epoch 3/10
95/95 [==============================] - 31s 322ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0073 - val_accuracy: 0.9974
Epoch 4/10
95/95 [==============================] - 29s 310ms/step - loss: 4.2151e-04 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 5/10
95/95 [==============================] - 29s 311ms/step - loss: 1.2037e-04 - accuracy: 1.0000 - val_loss: 8.9742e-04 - val_accuracy: 1.0000
Epoch 6/10
95/95 [==============================] - 29s 307ms/step - loss: 8.1183e-05 - accuracy: 1.0000 - val_loss: 8.1011e-04 - val_accuracy: 1.0000
Epoch 7/10
95/95 [==============================] - 35s 364ms/step - loss: 5.8657e-05 - accuracy: 1.0000 - val_loss: 7.73

In [17]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


24/24 [==============================] - 2s 82ms/step - loss: 7.6144e-04 - accuracy: 1.0000
Test Accuracy: 100.00%


In [18]:
# Speichern des Modells
model.save('model/fingerprint_model.h5')


In [2]:
import pymongo
from pymongo import MongoClient
import base64
import numpy as np
from PIL import Image
from io import BytesIO
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Verbindung zu MongoDB herstellen
client = MongoClient("mongodb://localhost:27017/")
db = client["fingerprintDB"]
collection = db["fingerprints"]

# Daten für User 2 und User 3 abrufen
user2_data = collection.find_one({"username": "username_2"})
user3_data = collection.find_one({"username": "username_3"})

# Funktion zum Dekodieren von Base64-Bildern
def decode_base64_image(base64_string):
    base64_data = base64_string.split(",")[1]
    byte_data = base64.b64decode(base64_data)
    image_data = BytesIO(byte_data)
    image = Image.open(image_data)
    return np.array(image)

# Funktion zum Abrufen der Canvases
def get_canvases(data):
    return [decode_base64_image(canvas) for canvas in data["canvases"]]

# Canvases für User 2 und User 3
user2_canvases = get_canvases(user2_data)
user3_canvases = get_canvases(user3_data)

# Positive Beispiele (User 2)
X_positive = np.array(user2_canvases)
y_positive = np.ones(len(X_positive))

# Negative Beispiele (User 3)
X_negative = np.array(user3_canvases)
y_negative = np.zeros(len(X_negative))

# Kombinieren der Daten
X = np.concatenate((X_positive, X_negative), axis=0)
y = np.concatenate((y_positive, y_negative), axis=0)

# Aufteilen der Daten in Trainings- und Testsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN-Modell erstellen
def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binäre Klassifikation
    return model

try:
    model.save('model/fingerprint_model.h5')
except Exception as e:
    print(f"An error occurred while saving the model: {str(e)}")
print("Model saved successfully.")

Model created.
Training the model...
Epoch 1/10
95/95 [==============================] - 17s 168ms/step - loss: 16.0010 - accuracy: 0.4918 - val_loss: 0.7032 - val_accuracy: 0.5171
Epoch 2/10
95/95 [==============================] - 15s 162ms/step - loss: 0.6794 - accuracy: 0.5697 - val_loss: 0.7252 - val_accuracy: 0.5118
Epoch 3/10
95/95 [==============================] - 15s 154ms/step - loss: 0.6472 - accuracy: 0.6191 - val_loss: 0.7240 - val_accuracy: 0.5158
Epoch 4/10
95/95 [==============================] - 15s 158ms/step - loss: 0.5977 - accuracy: 0.6770 - val_loss: 0.7886 - val_accuracy: 0.4882
Epoch 5/10
95/95 [==============================] - 15s 160ms/step - loss: 0.5427 - accuracy: 0.7240 - val_loss: 0.8070 - val_accuracy: 0.4895
Epoch 6/10
95/95 [==============================] - 15s 153ms/step - loss: 0.4834 - accuracy: 0.7572 - val_loss: 0.8702 - val_accuracy: 0.4934
Epoch 7/10
95/95 [==============================] - 16s 166ms/step - loss: 0.4059 - accuracy: 0.8151 - v

In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
from io import BytesIO
import base64

# Modell laden
model = load_model('model/fingerprint_model.h5')


In [10]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import base64
from io import BytesIO

def generate_random_canvas(txt):
    # Erstelle ein neues Bild mit RGBA-Modus
    canvas = Image.new('RGBA', (280, 35), (255, 255, 255, 0))
    draw = ImageDraw.Draw(canvas)
    
    # Schriftart und Größe festlegen
    font = ImageFont.truetype("arial.ttf", 18)
    
    # Text auf das Bild zeichnen
    draw.text((2, 15), txt, font=font, fill=(0, 102, 204, 255))
    draw.text((4, 19), txt, font=font, fill=(0, 102, 204, 179))
    draw.text((2, 23), txt, font=font, fill=(0, 102, 204, 255))
    draw.text((4, 27), txt, font=font, fill=(0, 102, 204, 179))
    draw.text((2, 31), txt, font=font, fill=(0, 102, 204, 255))
    draw.text((4, 35), txt, font=font, fill=(0, 102, 204, 179))
    
    # Bild in Base64 umwandeln
    buffered = BytesIO()
    canvas.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

# Beispiel-String
example_string = "example text"
canvas_image = generate_random_canvas(example_string)
print(canvas_image)  # Dies gibt die Base64-Darstellung des Canvas zurück


iVBORw0KGgoAAAANSUhEUgAAARgAAAAjCAYAAABPRBVWAAAHzElEQVR4nO2de5CXVRnHP4dcYttCkW5GGow2KKOBnmqjUenmoBUwEmo4TV4KDmNWNhMzXVYjYYrspuUMHGpsnImZki6AFSIzSRIqm68LhbRcWixZV2hLZEEZIE5/PM/Lvvz8Lfwcfi/uOs9n5jfn917O855zdp7ve57nnN+sSylhGIZRBoNe6QYYhvHqxQTGMIzSMIExDKM0TGAMwygNExjDMErDBMYwjNIwgTEMozRMYAzDKA0TGMMwSsMExjCM0jCBMQyjNExgDMMoDRMYwzBKwwTGMIzSMIExDKM0TGAMwygNExjDMErDBMYwjNI45ZVuwMnAhawd2JGi/0gdbJ0N/AC4NkW/74Qbd5LtG8bJxGYwL58bgMmAG6D2DeOkYQJjGEZpvCREciFrBuYC45G36F+A21P0D+n1y4EVwJIU/dWFeouAGcAVKfoHXMgc8DngOuA8fdY/gXuAO1L0SeutBx4H2oBbgDOBrcBsYANwJ3AF8ALwS2B2iv5Aoe56beNs4M1a57YU/apjdfx4/eyjzlJgih72uJAtTtF/ql7jBszqy75hDESOmsG4kH0QeBgYDHwW+AzwP2CVC9kUgBT9A8C9wFUuZBO13mWIkyzU6wDfBn4MrAGuQYRmFzAf+HRFO6YCXwJu12tDgSXAauA/wHTgN8AXgM9X1P2oPusufcZhYEXetor+LXQhm1dLP/ugBfi5fp8OdLqQDanjuFW1f4z2GEa/xuX/F0lnHH8HeoDxKfpDen4QsBY4AxiVok8uZMOAJ4F9wPuAJ4CDwNgU/T6t0wZsStFPP/KwkA0FdgKrUvST9dx64F3AmBR9u56bBSyg8LbX9m0BOlL0Ewt1xwKXp+hX6rkhet/zKfoL9Fw7sAPYBnQD02rpZ9UBC9k84OuIiFwJXK19PeFx68P+VSn6/X3/CQ2j/1IMkcYAo4FFwMUuZMX7

In [11]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
from io import BytesIO
import base64

# Modell laden
model = load_model('model/fingerprint_model.h5')

# Funktion zum Dekodieren von Base64-Bildern
def decode_base64_image(base64_string):
    byte_data = base64.b64decode(base64_string)
    image_data = BytesIO(byte_data)
    image = Image.open(image_data)
    return np.array(image)

# Funktion zum Vorhersagen
def predict_fingerprint(model, canvas_image):
    # Bild dekodieren
    image_array = decode_base64_image(canvas_image)
    # Normalisieren der Bilddaten
    image_array = image_array / 255.0
    # Hinzufügen einer Batch-Dimension
    image_array = np.expand_dims(image_array, axis=0)
    # Vorhersage
    prediction = model.predict(image_array)
    return prediction[0][0]

# Vorhersage durchführen
prediction = predict_fingerprint(model, canvas_image)
print(f"Prediction result: {prediction}")


1/1 [==============================] - 0s 267ms/step
Prediction result: 0.4938583970069885


In [12]:
import pymongo
from pymongo import MongoClient
import base64
import numpy as np
from PIL import Image
from io import BytesIO
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Verbindung zu MongoDB herstellen
client = MongoClient("mongodb://localhost:27017/")
db = client["fingerprintDB"]
collection = db["fingerprints"]

# Funktion zum Dekodieren von Base64-Bildern
def decode_base64_image(base64_string):
    base64_data = base64_string.split(",")[1]
    byte_data = base64.b64decode(base64_data)
    image_data = BytesIO(byte_data)
    image = Image.open(image_data)
    return np.array(image)

# Funktion zum Abrufen der Canvases
def get_canvases(data):
    return [decode_base64_image(canvas) for canvas in data["canvases"]]

# CNN-Modell erstellen
def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binäre Klassifikation
    return model

# Alle Benutzer abrufen
users = list(collection.find({}))

# Modelle für jeden Benutzer erstellen
for user in users:
    username = user['username']
    print(f"Creating model for {username}")

    # Positive Beispiele (Canvases des aktuellen Benutzers)
    user_canvases = get_canvases(user)
    X_positive = np.array(user_canvases)
    y_positive = np.ones(len(X_positive))

    # Negative Beispiele (Canvases der anderen Benutzer)
    X_negative = []
    y_negative = []

    for other_user in users:
        if other_user['username'] != username:
            other_user_canvases = get_canvases(other_user)
            X_negative.extend(other_user_canvases)
            y_negative.extend([0] * len(other_user_canvases))

    # Zufällig 2500 negative Beispiele auswählen
    if len(X_negative) > 2500:
        indices = np.random.choice(len(X_negative), 2500, replace=False)
        X_negative = np.array(X_negative)[indices]
        y_negative = np.array(y_negative)[indices]
    else:
        X_negative = np.array(X_negative)
        y_negative = np.array(y_negative)

    # Kombinieren der Daten
    X = np.concatenate((X_positive, X_negative), axis=0)
    y = np.concatenate((y_positive, y_negative), axis=0)

    # Aufteilen der Daten in Trainings- und Testsets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Modell erstellen und trainieren
    input_shape = X_train.shape[1:]
    model = create_cnn_model(input_shape)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

    # Modell speichern
    model_path = f'models/{username}_fingerprint_model.h5'
    try:
        model.save(model_path)
        print(f"Model for {username} saved successfully at {model_path}")
    except Exception as e:
        print(f"An error occurred while saving the model for {username}: {str(e)}")


Creating model for username_1
Epoch 1/10
110/110 [==============================] - 45s 396ms/step - loss: 8.3098 - accuracy: 0.8687 - val_loss: 0.0068 - val_accuracy: 0.9989
Epoch 2/10
110/110 [==============================] - 47s 428ms/step - loss: 0.0061 - accuracy: 0.9974 - val_loss: 0.0016 - val_accuracy: 0.9989
Epoch 3/10
110/110 [==============================] - 47s 423ms/step - loss: 1.0430e-04 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 0.9989
Epoch 4/10
110/110 [==============================] - 46s 419ms/step - loss: 3.2477e-05 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 0.9989
Epoch 5/10
110/110 [==============================] - 47s 424ms/step - loss: 1.9692e-05 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 0.9989
Epoch 6/10
110/110 [==============================] - 42s 382ms/step - loss: 1.3285e-05 - accuracy: 1.0000 - val_loss: 0.0013 - val_accuracy: 0.9989
Epoch 7/10
110/110 [==============================] - 39s 356ms/step - loss: 9.2194e